In [22]:
# example of calculating the frechet inception distance in Keras
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from keras.datasets import cifar10

In [2]:
# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

In [3]:
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [4]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
# define two fake collections of images

In [53]:
batch_size = 826 #this is for BiGAN
img_height = 217
img_width = 217

In [9]:
import tensorflow as tf
import keras

In [54]:
bigan_gen = keras.preprocessing.image.ImageDataGenerator()
bigan_data = bigan_gen.flow_from_directory(
  r'C:/Users/KIIT/AppData/Local/Programs/Python/Python36/Scripts/Untitled Folder 2/BiGAN Images/Classes', 
    target_size=(img_height, img_width), 
    batch_size=batch_size)


Found 10000 images belonging to 10 classes.


In [55]:
bigan_zeros = bigan_data.next()

In [23]:
(images1, _), (images2, _) = cifar10.load_data()

In [24]:
images1.shape

(50000, 32, 32, 3)

In [56]:
bigan_zeros = np.asarray(bigan_zeros[0])

In [57]:
bigan_zeros.shape #DONE!

(826, 217, 217, 3)

In [59]:
real_gen = keras.preprocessing.image.ImageDataGenerator()
real_data = real_data.flow_from_directory(
  r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=batch_size)

Found 8398 images belonging to 10 classes.


In [60]:
real_zeros = real_data.next()
real_zeros = np.asarray(real_zeros[0])
real_zeros.shape #DONE!

(826, 217, 217, 3)

In [61]:
print('Loaded', real_zeros.shape, bigan_zeros.shape)

Loaded (826, 217, 217, 3) (826, 217, 217, 3)


In [62]:
images1 = real_zeros
images2 = bigan_zeros

images1 = images1.astype('float32')
images2 = images2.astype('float32')

In [63]:
# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)

Scaled (826, 299, 299, 3) (826, 299, 299, 3)


In [64]:
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)

In [65]:
fid = calculate_fid(model, images1, images2)
print('FID: %.3f' % fid) #bigan zeros

FID: 133.675


In [71]:
import os
def get_image_counts(path1, path2):
    onlyfiles1 = next(os.walk(path1))[2]
    onlyfiles2 = next(os.walk(path2))[2]
    return len(onlyfiles1), len(onlyfiles2)
real_path = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set'

In [177]:
def zeros_zeros():
    
    bigan_gen = keras.preprocessing.image.ImageDataGenerator()
    bigan_data = bigan_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\0', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\0')))
    bigan_zeros = bigan_data.next()
    
    bigan_zeros = np.asarray(bigan_zeros[0])

    
    real_gen = keras.preprocessing.image.ImageDataGenerator()
    real_data = real_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\0', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\0')))
    
    #####
    
    real_zeros = real_data.next()
    real_zeros = np.asarray(real_zeros[0])
    
    print('Loaded', real_zeros.shape, bigan_zeros.shape)
    
    images1 = real_zeros
    images2 = bigan_zeros

    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    
    fid = calculate_fid(model, images1, images2)
    print('WGAN_Improved zeros FID: %.3f' % fid)
    
zeros_zeros()

Found 10055 images belonging to 10 classes.
Found 8398 images belonging to 10 classes.
Loaded (826, 217, 217, 3) (826, 217, 217, 3)
WGAN_Improved zeros FID: 110.077


In [178]:
def ones_ones():
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes'
    L = ['0']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
            
    #####
    
    bigan_gen = keras.preprocessing.image.ImageDataGenerator()
    bigan_data = bigan_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\1', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\1')))
    bigan_ones = bigan_data.next()
    
    bigan_ones = np.asarray(bigan_ones[0])
    
    #####
    
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set'
    L = ['0']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
    ######
    
    real_gen = keras.preprocessing.image.ImageDataGenerator()
    real_data = real_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\1', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\1')))
    
    #####
    
    real_ones = real_data.next()
    real_ones = np.asarray(real_ones[0])
    
    print('Loaded', real_ones.shape, bigan_ones.shape)
    
    images1 = real_ones
    images2 = bigan_ones

    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    
    fid = calculate_fid(model, images1, images2)
    print('WGAN_Improved ones FID: %.3f' % fid)
    
ones_ones()

Found 9179 images belonging to 9 classes.
Found 7572 images belonging to 9 classes.
Loaded (936, 217, 217, 3) (936, 217, 217, 3)
WGAN_Improved ones FID: 110.974


In [179]:
def twos_twos():
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes'
    L = ['1']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
            
    #####
    
    bigan_gen = keras.preprocessing.image.ImageDataGenerator()
    bigan_data = bigan_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\2', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\2')))
    bigan_twos = bigan_data.next()
    
    bigan_twos = np.asarray(bigan_twos[0])
    
    #####
    
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set'
    L = ['1']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
    ######
    
    real_gen = keras.preprocessing.image.ImageDataGenerator()
    real_data = real_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\2', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\2')))
    
    #####
    
    real_twos = real_data.next()
    real_twos = np.asarray(real_twos[0])
    
    print('Loaded', real_twos.shape, bigan_twos.shape)
    
    images1 = real_twos
    images2 = bigan_twos

    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    
    fid = calculate_fid(model, images1, images2)
    print('WGAN_Improved twos FID: %.3f' % fid)
    
twos_twos()

Found 8156 images belonging to 8 classes.
Found 6636 images belonging to 8 classes.
Loaded (835, 217, 217, 3) (835, 217, 217, 3)
WGAN_Improved twos FID: 124.061


In [180]:
def threes_threes():
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes'
    L = ['2']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
            
    #####
    
    bigan_gen = keras.preprocessing.image.ImageDataGenerator()
    bigan_data = bigan_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\3', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\3')))
    bigan_threes = bigan_data.next()
    
    bigan_threes = np.asarray(bigan_threes[0])
    
    #####
    
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set'
    L = ['2']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
    ######
    
    real_gen = keras.preprocessing.image.ImageDataGenerator()
    real_data = real_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\3', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\3')))
    
    #####
    
    real_threes = real_data.next()
    real_threes = np.asarray(real_threes[0])
    
    print('Loaded', real_threes.shape, bigan_threes.shape)
    
    images1 = real_threes
    images2 = bigan_threes

    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    
    fid = calculate_fid(model, images1, images2)
    print('WGAN_Improved threes FID: %.3f' % fid)
    
threes_threes()

Found 7027 images belonging to 7 classes.
Found 5801 images belonging to 7 classes.
Loaded (870, 217, 217, 3) (870, 217, 217, 3)
WGAN_Improved threes FID: 115.370


In [181]:
def fours_fours():
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes'
    L = ['3']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
            
    #####
    
    bigan_gen = keras.preprocessing.image.ImageDataGenerator()
    bigan_data = bigan_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\4', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\4')))
    bigan_fours = bigan_data.next()
    
    bigan_fours = np.asarray(bigan_fours[0])
    
    #####
    
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set'
    L = ['3']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
    ######
    
    real_gen = keras.preprocessing.image.ImageDataGenerator()
    real_data = real_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\4', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\4')))
    
    #####
    
    real_fours = real_data.next()
    real_fours = np.asarray(real_fours[0])
    
    print('Loaded', real_fours.shape, bigan_fours.shape)
    
    images1 = real_fours
    images2 = bigan_fours

    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    
    fid = calculate_fid(model, images1, images2)
    print('WGAN_Improved fours FID: %.3f' % fid)
    
fours_fours()

Found 5836 images belonging to 6 classes.
Found 4931 images belonging to 6 classes.
Loaded (814, 217, 217, 3) (814, 217, 217, 3)
WGAN_Improved fours FID: 112.208


In [182]:
def fives_fives():
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes'
    L = ['4']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
            
    #####
    
    bigan_gen = keras.preprocessing.image.ImageDataGenerator()
    bigan_data = bigan_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\5', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\5')))
    bigan_fives = bigan_data.next()
    
    bigan_fives = np.asarray(bigan_fives[0])
    
    #####
    
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set'
    L = ['4']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
    ######
    
    real_gen = keras.preprocessing.image.ImageDataGenerator()
    real_data = real_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\5', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\5')))
    
    #####
    
    real_fives = real_data.next()
    real_fives = np.asarray(real_fives[0])
    
    print('Loaded', real_fives.shape, bigan_fives.shape)
    
    images1 = real_fives
    images2 = bigan_fives

    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    
    fid = calculate_fid(model, images1, images2)
    print('WGAN_Improved fives FID: %.3f' % fid)
    
fives_fives()

Found 4975 images belonging to 5 classes.
Found 4117 images belonging to 5 classes.
Loaded (759, 217, 217, 3) (759, 217, 217, 3)
WGAN_Improved fives FID: 107.879


In [183]:
def sixes_sixes():
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes'
    L = ['5']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
            
    #####
    
    bigan_gen = keras.preprocessing.image.ImageDataGenerator()
    bigan_data = bigan_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\6', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\6')))
    bigan_sixes = bigan_data.next()
    
    bigan_sixes = np.asarray(bigan_sixes[0])
    
    #####
    
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set'
    L = ['5']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
    ######
    
    real_gen = keras.preprocessing.image.ImageDataGenerator()
    real_data = real_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\6', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\6')))
    
    #####
    
    real_sixes = real_data.next()
    real_sixes = np.asarray(real_sixes[0])
    
    print('Loaded', real_sixes.shape, bigan_sixes.shape)
    
    images1 = real_sixes
    images2 = bigan_sixes

    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    
    fid = calculate_fid(model, images1, images2)
    print('WGAN_Improved sixes FID: %.3f' % fid)
    
sixes_sixes()

Found 4144 images belonging to 4 classes.
Found 3358 images belonging to 4 classes.
Loaded (827, 217, 217, 3) (827, 217, 217, 3)
WGAN_Improved sixes FID: 104.845


In [184]:
def sevens_sevens():
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes'
    L = ['6']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
            
    #####
    
    bigan_gen = keras.preprocessing.image.ImageDataGenerator()
    bigan_data = bigan_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\7', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\7')))
    bigan_sevens = bigan_data.next()
    
    bigan_sevens = np.asarray(bigan_sevens[0])
    
    #####
    
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set'
    L = ['6']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
    ######
    
    real_gen = keras.preprocessing.image.ImageDataGenerator()
    real_data = real_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\7', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\7')))
    
    #####
    
    real_sevens = real_data.next()
    real_sevens = np.asarray(real_sevens[0])
    
    print('Loaded', real_sevens.shape, bigan_sevens.shape)
    
    images1 = real_sevens
    images2 = bigan_sevens

    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    
    fid = calculate_fid(model, images1, images2)
    print('WGAN_Improved sevens FID: %.3f' % fid)
    
sevens_sevens()

Found 3043 images belonging to 3 classes.
Found 2531 images belonging to 3 classes.
Loaded (880, 217, 217, 3) (880, 217, 217, 3)
WGAN_Improved sevens FID: 113.610


In [185]:
def eights_eights():
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes'
    L = ['7']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
            
    #####
    
    bigan_gen = keras.preprocessing.image.ImageDataGenerator()
    bigan_data = bigan_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\8', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\8')))
    bigan_eights = bigan_data.next()
    
    bigan_eights = np.asarray(bigan_eights[0])
    
    #####
    
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set'
    L = ['7']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
    ######
    
    real_gen = keras.preprocessing.image.ImageDataGenerator()
    real_data = real_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\8', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\8')))
    
    #####
    
    real_eights = real_data.next()
    real_eights = np.asarray(real_eights[0])
    
    print('Loaded', real_eights.shape, bigan_eights.shape)
    
    images1 = real_eights
    images2 = bigan_eights

    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    
    fid = calculate_fid(model, images1, images2)
    print('WGAN_Improved eights FID: %.3f' % fid)
    
eights_eights()

Found 2127 images belonging to 2 classes.
Found 1651 images belonging to 2 classes.
Loaded (813, 217, 217, 3) (813, 217, 217, 3)
WGAN_Improved eights FID: 133.387


In [186]:
def nines_nines():
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes'
    L = ['8']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
            
    #####
    
    bigan_gen = keras.preprocessing.image.ImageDataGenerator()
    bigan_data = bigan_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\9', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\9')))
    bigan_nines = bigan_data.next()
    
    bigan_nines = np.asarray(bigan_nines[0])
    
    #####
    
    cur_dir = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set'
    L = ['8']

    list_dir = os.listdir(cur_dir)
    dest = r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\reserve'

    for sub_dir in list_dir:
        if sub_dir in L:
            dir_to_move = os.path.join(cur_dir, sub_dir)
            shutil.move(dir_to_move, dest)
    ######
    
    real_gen = keras.preprocessing.image.ImageDataGenerator()
    real_data = real_gen.flow_from_directory(r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\MNIST as jpg\test_set', 
    target_size=(img_height, img_width), 
    batch_size=min(get_image_counts(real_path+r'\9', r'C:\Users\KIIT\AppData\Local\Programs\Python\Python36\Scripts\Untitled Folder 2\WGAN_Improved\Classes\9')))
    
    #####
    
    real_nines = real_data.next()
    real_nines = np.asarray(real_nines[0])
    
    print('Loaded', real_nines.shape, bigan_nines.shape)
    
    images1 = real_nines
    images2 = bigan_nines

    images1 = images1.astype('float32')
    images2 = images2.astype('float32')
    images1 = scale_images(images1, (299,299,3))
    images2 = scale_images(images2, (299,299,3))
    
    # pre-process images
    images1 = preprocess_input(images1)
    images2 = preprocess_input(images2)
    
    fid = calculate_fid(model, images1, images2)
    print('WGAN_Improved nines FID: %.3f' % fid)
    
nines_nines()

Found 1007 images belonging to 1 classes.
Found 838 images belonging to 1 classes.
Loaded (838, 217, 217, 3) (838, 217, 217, 3)
WGAN_Improved nines FID: 115.865
